# Exploratory Data Analysis: Energy Consumption For Chicago In 2010

## Life cycle of machine learning project:
1. Understanding the problem
1. Data collection
1. Data checks to perform 
1. Exploratory data analysis
1. Data pre-preocessing
1. Model Training
1. Choose best model

### 1: Understanding the problem
The point of this project is to show how energy consumption is affected by variables such as time of the year, type of building, size of building, number of people, etc.

### 2: Data Collection
- Dataset <a href='https://data.cityofchicago.org/Environment-Sustainable-Development/Energy-Usage-2010/8yq3-m6wp/about_data'>source</a>


### 2.1: Import data and required packages

In [10]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

### Read and print 5 first rows of dataset

In [12]:
df = pd.read_csv('data/chicago_energy_consumption.csv')
df.head()

,COMMUNITY AREA NAME,CENSUS BLOCK,BUILDING TYPE,BUILDING_SUBTYPE,KWH JANUARY 2010,KWH FEBRUARY 2010,KWH MARCH 2010,KWH APRIL 2010,KWH MAY 2010,KWH JUNE 2010,...,TOTAL POPULATION,TOTAL UNITS,AVERAGE STORIES,AVERAGE BUILDING AGE,AVERAGE HOUSESIZE,OCCUPIED UNITS,OCCUPIED UNITS PERCENTAGE,RENTER-OCCUPIED HOUSING UNITS,RENTER-OCCUPIED HOUSING PERCENTAGE,OCCUPIED HOUSING UNITS
0,Archer Heights,1.703157e+14,Residential,Multi < 7,NaN,NaN,NaN,NaN,NaN,NaN,...,89.0,24.0,2.0,71.33,3.87,23.0,0.9582,9.0,0.3910,23.0
1,Ashburn,1.703170e+14,Residential,Multi 7+,7334.0,7741.0,4214.0,4284.0,2518.0,4273.0,...,112.0,67.0,2.0,41.00,1.81,62.0,0.9254,50.0,0.8059,62.0
2,Auburn Gresham,1.703171e+14,Commercial,Multi < 7,NaN,NaN,NaN,NaN,NaN,NaN,...,102.0,48.0,3.0,86.00,3.00,34.0,0.7082,23.0,0.6759,34.0
3,Austin,1.703125e+14,Commercial,Multi < 7,NaN,NaN,NaN,NaN,NaN,NaN,...,121.0,56.0,2.0,84.00,2.95,41.0,0.7321,32.0,0.7800,41.0
4,Austin,1.703125e+14,Commercial,Multi < 7,NaN,NaN,NaN,NaN,NaN,NaN,...,62.0,23.0,2.0,85.00,3.26,19.0,0.8261,11.0,0.5790,19.0


### Shape of dataset

In [23]:
print('Columns: ',df.shape[1])
print('Rows: ',df.shape[0])

Columns:  73
Rows:  67051


### 2.2: Dataset information

In [27]:
df.columns.values

array(['COMMUNITY AREA NAME', 'CENSUS BLOCK', 'BUILDING TYPE',
       'BUILDING_SUBTYPE', 'KWH JANUARY 2010', 'KWH FEBRUARY 2010',
       'KWH MARCH 2010', 'KWH APRIL 2010', 'KWH MAY 2010',
       'KWH JUNE 2010', 'KWH JULY 2010', 'KWH AUGUST 2010',
       'KWH SEPTEMBER 2010', 'KWH OCTOBER 2010', 'KWH NOVEMBER 2010',
       'KWH DECEMBER 2010', 'TOTAL KWH', 'ELECTRICITY ACCOUNTS',
       'ZERO KWH ACCOUNTS', 'THERM JANUARY 2010', 'THERM FEBRUARY 2010',
       'THERM MARCH 2010', 'TERM APRIL 2010', 'THERM MAY 2010',
       'THERM JUNE 2010', 'THERM JULY 2010', 'THERM AUGUST 2010',
       'THERM SEPTEMBER 2010', 'THERM OCTOBER 2010',
       'THERM NOVEMBER 2010', 'THERM DECEMBER 2010', 'TOTAL THERMS',
       'GAS ACCOUNTS', 'KWH TOTAL SQFT', 'THERMS TOTAL SQFT',
       'KWH MEAN 2010', 'KWH STANDARD DEVIATION 2010', 'KWH MINIMUM 2010',
       'KWH 1ST QUARTILE 2010', 'KWH 2ND QUARTILE 2010',
       'KWH 3RD QUARTILE 2010', 'KWH MAXIMUM 2010', 'KWH SQFT MEAN 2010',
       'KWH SQFT STAND

### Dataset description
Click <a href='data_description.md'>here</a> for data description